# Cryptocurrencies Analysis

### Importing libraries:

In [1]:
import re
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import matplotlib
import matplotlib.pyplot as plt

### Importing data from Wikipedia - List of cryptocurrencies

In [2]:
#Extracting the information from the web page.
res = requests.get("https://en.wikipedia.org/wiki/List_of_cryptocurrencies")
soup = BeautifulSoup(res.text, 'html.parser')

In [3]:
datos=soup.find_all('table')[0]
datos.find_all('tr')[1].find_all("td")[0]

<td>2009
</td>

In [4]:
def createDF(table):
    
    headers = [header.text[:-1] for header in datos.find_all('th')]
    data=[]
    for row in table.find_all('tr')[1:]:
        currency = {}
        for index,value in enumerate(row.find_all("td")):
            currency[headers[index]]=value.text[:-1]
        data.append(currency)
    data = pd.DataFrame(data)
    return data

wiki=createDF(datos)

In [5]:
def quitarBrackets():
    for columna in wiki:
        wiki[columna] = wiki[columna].str.replace(r"\[.*\]","")
    return wiki
clean_wiki = quitarBrackets()

In [6]:
def changeWiki():    
    for i in range(len(clean_wiki)):

        diccionario = {'EOS.IO':'EOS',
                       'Bitcoin-cash':'Bitcoin Cash', 
                       'Ether or "Ethereum"':'Ethereum'
                      }
        for x,y in diccionario.items(): 
                clean_wiki['Currency'].values[i] = clean_wiki['Currency'].values[i].replace(x, y)
    return clean_wiki

clean_wiki=changeWiki()

### Importing data from Kaggle - Historical data

In [7]:
historical_data = pd.read_csv("INPUT/consolidated_coin_data.csv",encoding='latin1')

In [8]:
def changeKaggle():    
    for i in range(len(historical_data)):

        diccionario = {'bitcoin':'Bitcoin',
                       'bitcoin-cash':'Bitcoin Cash', 
                       'eos':'EOS',
                       'ethereum':'Ethereum',
                       'litecoin':'Litecoin',
                       'stellar':'Stellar',
                       'tether':'Tether',
                       'xrp':'Ripple'
                      }
        for x,y in diccionario.items(): 
                historical_data['Currency'].values[i] = historical_data['Currency'].values[i].replace(x, y)
    return historical_data

clean_kaggle=changeKaggle()

# CONCLUSIONS - Merging both files

In [9]:
def merging():
    result=pd.merge(clean_wiki,
                   clean_kaggle,
                   on='Currency')
    return result[['Release','Currency','Symbol','Programming language of implementation','Notes','Date','Close','Market Cap']]
final_table=merging()

In [11]:
#Exporting the file.
final_table.to_csv("OUTPUT/final_table.csv")

In [14]:
#Casting the columns that should be as float.
final_table['Close'] = final_table['Close'].str.replace(',', '').astype(float)
final_table['Market Cap']=final_table['Market Cap'].str.replace(',', '').astype(float)
final_table['Date']=pd.to_datetime(final_table['Date'])